### Monitoring memory in python

In [80]:
import psutil

In [81]:
# psutil.virtual_memory?

In [82]:
vmem = psutil.virtual_memory()
vmem

svmem(total=8369946624, available=4382461952, percent=47.6, used=3537788928, free=1636704256, active=3320774656, inactive=2263658496, buffers=227864576, cached=2967588864, shared=137945088, slab=540848128)

In [83]:
total_mem_bytes = vmem.total
total_mem_bytes

8369946624

In [84]:
avail_mem_bytes = vmem.available
avail_mem_bytes

4382461952

In [7]:
from ezenum import StringEnum
LINETYPE = StringEnum(['Time', 'User', 'Content', 'Other'])

In [45]:
def interpret_line(decoded_line):
    try:
        char1 = decoded_line[1]
        if char1 == '\t':
            char0 = decoded_line[0]
            if char0 == 'T':
                return LINETYPE.Time, decoded_line[2:-1]
            elif char0 == 'U':
                return LINETYPE.User, decoded_line[21:-1]
            elif char0 == 'W':
                return LINETYPE.Content, decoded_line[2:-1]
            else:
                return LINETYPE.Other, decoded_line[2:-1]
        else:
            return LINETYPE.Other, ''
    except IndexError:
        return LINETYPE.Other, ''

In [112]:
import gzip
from time import time

NO_CONTENT = 'No Post Title'
user_to_tweets_str = {}
# user_to_tweets_count = {}
monitor_line_freq = 1000000
min_avail_mem_bytes = 500 * 1000000
report_template = (
    '{:.2f} min running | {} lines processed | ~ {} tweets processed |'
    ' {} tpm | {} available memory'
)


def merge_user_tweets_in_file(fpath, line_limit):
    most_recent_user = None
    starting_available_mem = psutil.virtual_memory().available
    start_time = time()
    with gzip.open(fpath, 'r') as textf:
        for i, line in enumerate(textf):
            try:
                decoded_line = line.decode('utf-8')
                ltype, lcontent = interpret_line(decoded_line)
                if ltype == LINETYPE.User:
                    most_recent_user = lcontent
                elif ltype == LINETYPE.Content and lcontent != NO_CONTENT:
                    user_to_tweets_str[most_recent_user] = user_to_tweets_str.get(
                        most_recent_user, '') + ' ' + lcontent
#                     user_to_tweets_count[most_recent_user] = user_to_tweets_count.get(most_recent_user, 0) + 1
            except Exception as e:
                print(decoded_line)
                print(interpret_line(decoded_line))
                raise e
            if i % monitor_line_freq == 0:
                av_mem = psutil.virtual_memory().available
                seconds_running = time() - start_time
                report = report_template.format(
                    seconds_running / 60,
                    i,
                    i / 4,
                    (i / 4) / (seconds_running / 60),
                    av_mem,
                )
                print(report, end='\r')
                if av_mem < min_avail_mem_bytes:
                    return
#             if i >= line_limit:
#                 break

In [113]:
fpath1 = '/home/shaypalachy/data/twitter7/tweets2009-06.txt.gz'

In [114]:
merge_user_tweets_in_file(fpath1, 10000)

KeyboardInterrupt: 

In [ ]:
# user_to_tweets_str

In [116]:
len(user_to_tweets_str)

171753

In [76]:
user_to_tweets_str['poluakerford']

' Student murder sentences examined http://tinyurl.com/mkzt27 Surveys: More than 2M households unready for DTV (AP) http://tinyurl.com/naewb8'

In [ ]:
# user_to_tweets_count

In [77]:
import sys

In [78]:
sys.getsizeof(user_to_tweets_str)

73824

In [79]:
sys.getsizeof(user_to_tweets_count)

73824